In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objects as go

In [3]:
# Reading data 
prices_df = pd.read_csv('prices.csv')
prices_df['date'] = pd.to_datetime(prices_df['date'])

In [4]:
#Calculating RSI and implementing stoploss strategy

def calculate_rsi(data, period=14, overbought=70, oversold=30, stop_loss_pct=5):
    def calculate_rsi_group(group):
        delta = group['close'].diff()
        gains = delta.mask(delta < 0, 0)
        losses = -delta.mask(delta > 0, 0)
        avg_gains = gains.rolling(window=period).mean()
        avg_losses = losses.rolling(window=period).mean()
        rs = avg_gains / avg_losses
        rsi = 100 - (100 / (1 + rs))
        group['RSI'] = rsi

        group['Signal'] = 'Hold'
        
        below_oversold = (group['RSI'] < oversold) & (group['RSI'].shift(1) < oversold)  
        above_overbought = (group['RSI'] > overbought) & (group['RSI'].shift(1) > overbought) 

        remaining_hold = (below_oversold) & (group['RSI'] < oversold)
        
        group.loc[group['RSI'] < oversold, 'Signal'] = 'Buy'
        group.loc[below_oversold, 'Signal'] = 'Hold' 
        group.loc[group['RSI'] > overbought, 'Signal'] = 'Sell'
        group.loc[above_overbought, 'Signal'] = 'Hold'

        
         # Calculate the stop-loss price for each "Buy" signal
        stop_loss_price = group['close'] * (1 - stop_loss_pct / 100)
        group['StopLoss'] = stop_loss_price.where(group['Signal'] == 'Buy')
        
        # Condition for stop-loss triggered
        stop_loss_triggered = (group['RSI'] < oversold) & (group['RSI'].shift(1) < oversold) & (group['low'] < group['StopLoss'])
        group.loc[stop_loss_triggered, 'Signal'] = 'Sell_StopLoss'

        return group

    rsi_signals_df = data.groupby('ticker').apply(calculate_rsi_group).reset_index(drop=True)

    return rsi_signals_df

signals_df = calculate_rsi(prices_df)

In [5]:
def calculate_portfolio_value_rsi(group, initial_capital):
    group['Shares'] = 0
    group['Holding_Shares'] = 0
    group['Equity_Capital'] = 0
    group['Capital'] = initial_capital
    group['Portfolio_Value'] = initial_capital
    
    invested_capital = initial_capital
    holding_shares = 0
    
    for i, row in group.iterrows():
        if row['Signal'] == 'Buy':
            num_shares_to_buy = invested_capital // row['close']
            holding_shares += num_shares_to_buy
            invested_capital -= num_shares_to_buy * row['close']
        elif row['Signal'] == 'Sell':
            invested_capital += holding_shares * row['close']
            holding_shares = 0
         
        equity_capital = invested_capital + (holding_shares * row['close'])
        group.at[i, 'Shares'] = holding_shares
        group.at[i, 'Holding_Shares'] = holding_shares
        group.at[i, 'Equity_Capital'] = equity_capital
        group.at[i, 'Capital'] = invested_capital
        group.at[i, 'Portfolio_Value'] = equity_capital
        
    return group

In [6]:
portfolio_rsi_df = calculate_portfolio_value_rsi(signals_df, 100000)

In [7]:
#Calculating daily returns

def calculate_daily_returns(data):
    data['Return_Percentage'] = (data['Portfolio_Value'] / data['Portfolio_Value'].shift(1) - 1) * 100
    return data

In [8]:
def calculate_cagr(data):
    final_portfolio_value = data['Portfolio_Value']
    initial_portfolio_value = data['Portfolio_Value'].iloc[0]
    # num_years = pd.to_datetime(prices['date']) - pd.to_datetime(prices['date'].loc[0])
    # num_years = num_years/np.timedelta64(1, 'Y')
    num_years = (pd.to_datetime(data['date']) - pd.to_datetime(data['date'].iloc[0])).dt.days / 365.25
    data['num_years'] = num_years

    cagr = (final_portfolio_value / initial_portfolio_value) ** (1 / num_years) - 1
    
    data['CAGR'] = cagr
    
    return data    

In [9]:
# Calculating drawdown percentage 
def calculate_drawdown(data):

    data['Peak'] = data['Portfolio_Value'].cummax()
    data['Drawdown'] = ((data['Peak'] - data['Portfolio_Value']) / data['Peak'])

    return data

In [10]:
#portfolio_rsi_df = portfolio_rsi_df[portfolio_rsi_df['Signal'] != 'Hold']
portfolio_rsi_df = calculate_daily_returns(portfolio_rsi_df)
portfolio_rsi_df = calculate_cagr(portfolio_rsi_df)
portfolio_rsi_df = calculate_drawdown(portfolio_rsi_df)

In [11]:
portfolio_rsi_df

,ticker,date,open,high,low,close,RSI,Signal,StopLoss,Shares,Holding_Shares,Equity_Capital,Capital,Portfolio_Value,Return_Percentage,num_years,CAGR,Peak,Drawdown
0,AXISBANK,2010-01-04,198.779999,199.990005,197.619995,140.712021,NaN,Hold,NaN,0,0,1.000000e+05,1.000000e+05,1.000000e+05,NaN,0.000000,0.000000,1.000000e+05,0.000000
1,AXISBANK,2010-01-05,199.600006,204.000000,199.139999,143.647934,NaN,Hold,NaN,0,0,1.000000e+05,1.000000e+05,1.000000e+05,0.000000,0.002738,0.000000,1.000000e+05,0.000000
2,AXISBANK,2010-01-06,204.000000,205.740005,198.509995,141.236801,NaN,Hold,NaN,0,0,1.000000e+05,1.000000e+05,1.000000e+05,0.000000,0.005476,0.000000,1.000000e+05,0.000000
3,AXISBANK,2010-01-07,199.289993,201.199997,197.820007,142.236725,NaN,Hold,NaN,0,0,1.000000e+05,1.000000e+05,1.000000e+05,0.000000,0.008214,0.000000,1.000000e+05,0.000000
4,AXISBANK,2010-01-08,200.600006,203.979996,199.270004,144.094727,NaN,Hold,NaN,0,0,1.000000e+05,1.000000e+05,1.000000e+05,0.000000,0.010951,0.000000,1.000000e+05,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47568,SRTRANSFIN,2020-06-09,636.900024,650.950012,613.000000,619.599976,59.987710,Hold,NaN,4845,4845,3.002295e+06,3.332546e+02,3.002295e+06,-1.923024,10.428474,0.385718,8.711905e+06,0.655380
47569,SRTRANSFIN,2020-06-10,622.599976,641.700012,612.099976,635.000000,66.548886,Hold,NaN,4845,4845,3.076908e+06,3.332546e+02,3.076908e+06,2.485203,10.431211,0.388864,8.711905e+06,0.646816
47570,SRTRANSFIN,2020-06-11,615.250000,676.000000,615.250000,637.900024,68.600187,Hold,NaN,4845,4845,3.090959e+06,3.332546e+02,3.090959e+06,0.456647,10.433949,0.389351,8.711905e+06,0.645203
47571,SRTRANSFIN,2020-06-12,615.000000,693.700012,606.000000,684.400024,89.451931,Sell,NaN,0,0,3.316251e+06,3.316251e+06,3.316251e+06,7.288758,10.436687,0.398628,8.711905e+06,0.619343


In [67]:
def create_single_ticker_dataframe(data, ticker):
    single_ticker_df = data[data['ticker'] == ticker].copy()

    return single_ticker_df


ticker_to_analyze = 'SBILIFE'
single_ticker_dataframe = create_single_ticker_dataframe(portfolio_rsi_df, ticker_to_analyze)


single_ticker_dataframe

,ticker,date,open,high,low,close,RSI,Signal,StopLoss,Shares,Holding_Shares,Equity_Capital,Capital,Portfolio_Value,Return_Percentage,num_years,CAGR,Peak,Drawdown
41767,SBILIFE,2017-10-03,735.000000,740.000000,701.700012,703.133728,NaN,Hold,NaN,0,0,1.662496e+06,1.662496e+06,1.662496e+06,0.0,7.745380,0.437512,8.711905e+06,0.809170
41768,SBILIFE,2017-10-04,710.150024,710.750000,699.000000,699.258118,NaN,Hold,NaN,0,0,1.662496e+06,1.662496e+06,1.662496e+06,0.0,7.748118,0.437328,8.711905e+06,0.809170
41769,SBILIFE,2017-10-05,704.549988,704.549988,686.099976,690.214905,NaN,Hold,NaN,0,0,1.662496e+06,1.662496e+06,1.662496e+06,0.0,7.750856,0.437143,8.711905e+06,0.809170
41770,SBILIFE,2017-10-06,693.900024,702.000000,689.000000,692.401184,NaN,Hold,NaN,0,0,1.662496e+06,1.662496e+06,1.662496e+06,0.0,7.753593,0.436959,8.711905e+06,0.809170
41771,SBILIFE,2017-10-09,694.099976,702.500000,694.099976,694.289307,NaN,Hold,NaN,0,0,1.662496e+06,1.662496e+06,1.662496e+06,0.0,7.761807,0.436408,8.711905e+06,0.809170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42423,SBILIFE,2020-06-09,782.099976,789.000000,752.500000,758.500000,65.091006,Hold,NaN,0,0,1.319966e+06,1.319966e+06,1.319966e+06,0.0,10.428474,0.280714,8.711905e+06,0.848487
42424,SBILIFE,2020-06-10,759.000000,760.000000,739.700012,741.700012,56.888304,Hold,NaN,0,0,1.319966e+06,1.319966e+06,1.319966e+06,0.0,10.431211,0.280631,8.711905e+06,0.848487
42425,SBILIFE,2020-06-11,740.000000,757.599976,735.000000,740.900024,53.013778,Hold,NaN,0,0,1.319966e+06,1.319966e+06,1.319966e+06,0.0,10.433949,0.280548,8.711905e+06,0.848487
42426,SBILIFE,2020-06-12,730.000000,760.900024,728.750000,750.450012,55.591928,Hold,NaN,0,0,1.319966e+06,1.319966e+06,1.319966e+06,0.0,10.436687,0.280465,8.711905e+06,0.848487


In [13]:
pyo.plot([{
 'x': single_ticker_dataframe.date,
 'y': single_ticker_dataframe.Return_Percentage 
}
])

'temp-plot.html'

In [14]:
portfolio_rsi_df.loc[2557:2574, ['ticker','date','close','num_years','RSI','Portfolio_Value','Peak','Drawdown','CAGR']]

,ticker,date,close,num_years,RSI,Portfolio_Value,Peak,Drawdown,CAGR
2557,AXISBANK,2020-05-26,341.299988,10.390144,34.933665,130193.592087,258539.327255,0.496426,2.571967e-02
2558,AXISBANK,2020-05-27,387.000000,10.392882,49.546233,147605.296738,258539.327255,0.429080,3.817589e-02
2559,AXISBANK,2020-05-28,390.950012,10.395619,48.394382,149110.251389,258539.327255,0.423259,3.917920e-02
2560,AXISBANK,2020-05-29,384.950012,10.398357,50.763164,146824.251389,258539.327255,0.432101,3.762585e-02
2561,AXISBANK,2020-06-01,396.950012,10.406571,54.360908,151396.251389,258539.327255,0.414417,4.065752e-02
2562,AXISBANK,2020-06-02,410.100006,10.409309,55.661067,156406.399063,258539.327255,0.395038,4.390653e-02
2563,AXISBANK,2020-06-03,409.549988,10.412047,48.755241,156196.842087,258539.327255,0.395849,4.376033e-02
2564,AXISBANK,2020-06-04,394.350006,10.414784,47.910938,150405.649063,258539.327255,0.418248,3.996910e-02
2565,AXISBANK,2020-06-05,405.299988,10.417522,54.667039,154577.592087,258539.327255,0.402112,4.269328e-02
2566,AXISBANK,2020-06-08,430.250000,10.425736,70.455204,164083.546738,258539.327255,0.365344,4.864450e-02


In [15]:
# Choose a single ticker
selected_ticker = 'AXISBANK'

# Filter data for the selected ticker
selected_ticker_data = portfolio_rsi_df[portfolio_rsi_df['ticker'] == selected_ticker]

# Export data to Excel for the selected ticker
data_to_export = selected_ticker_data[['date', 'close', 'Portfolio_Value','num_years']]
data_to_export.to_excel(f'data_for_{selected_ticker}_NEW.xlsx', index=False)

In [16]:
portfolio_rsi_df['Drawdown'].unique()

array([0.        , 0.00437486, 0.00658532, ..., 0.64681569, 0.64520288,
       0.61934258])

In [78]:
portfolio_rsi_df.loc[40013:40043, ['ticker','Drawdown','CAGR']]

,ticker,Drawdown,CAGR
40013,PNB,0.564636,2.010256
40014,PNB,0.560948,2.012444
40015,PNB,0.560948,2.009696
40016,PNB,0.560948,2.001492
40017,PNB,0.560948,1.998771
40018,PNB,0.560948,1.993350
40019,PNB,0.560948,1.990650
40020,PNB,0.560948,1.982592
40021,PNB,0.560948,1.979919
40022,PNB,0.560948,1.977253
